In [83]:
# ===============================================================
# Imports
# ===============================================================

import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score

from sklearn.svm import SVC

from sklearn.base import TransformerMixin, BaseEstimator

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


In [84]:
# ===============================================================
# Load data
# ===============================================================

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
train = train.dropna(subset=["spend_category"])#some columns had null!

# y.isna().sum()
# y.shape


In [85]:
train["spend_category"].value_counts()

spend_category
0.0    6245
1.0    4911
2.0    1464
Name: count, dtype: int64

In [86]:
train.drop_duplicates(inplace=True)
train

,trip_id,country,age_group,travel_companions,num_females,num_males,main_activity,visit_purpose,is_first_visit,mainland_stay_nights,...,food_included,domestic_transport_included,sightseeing_included,guide_included,insurance_included,days_booked_before_trip,arrival_weather,total_trip_days,has_special_requirements,spend_category
0,tour_idftaa27vp,FRANCE,45-64,With Spouse and Children,1.0,2.0,Beach Tourism,Leisure and Holidays,Yes,0,...,No,No,No,No,No,NaN,"cloudy,",30+,NaN,1.0
1,tour_iduck75m57,KENYA,45-64,Alone,1.0,0.0,Conference Tourism,Meetings and Conference,Yes,6,...,No,No,No,No,No,15-30,"sunny,",30+,NaN,2.0
2,tour_id8y3w40h8,SOUTH AFRICA,25-44,With Other Friends/Relatives,2.0,0.0,Cultural Tourism,Meetings and Conference,No,4,...,No,No,No,No,No,90+,"sunny,",30+,none,2.0
3,tour_idkoh8mkgr,ITALY,25-44,With Spouse,1.0,1.0,Widlife Tourism,Leisure and Holidays,Yes,0,...,Yes,Yes,Yes,Yes,No,8-14,NaN,NaN,none,0.0
4,tour_idkmsfa00a,ITALY,25-44,With Spouse,1.0,1.0,Beach Tourism,Leisure and Holidays,Yes,0,...,Yes,No,No,No,No,90+,"sunny,",7-14,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12649,tour_idlln8s2r3,NETHERLANDS,25-44,With Other Friends/Relatives,0.0,2.0,Wildlife Tourism,Business,No,7,...,Yes,Yes,Yes,No,No,NaN,Stormy,30+,NaN,0.0
12650,tour_id3ta5n81h,KENYA,25-44,Alone,1.0,0.0,Hunting Tourism,Meetings and Conference,No,10,...,No,No,No,No,No,61-90,humid,7-14,NaN,1.0
12651,tour_id88nvd60z,UNITED STATES OF AMERICA,45-64,Alone,0.0,1.0,Widlife Tourism,Scientific and Academic,No,9,...,No,No,No,No,No,90+,"sunny,",1-6,NaN,1.0
12652,tour_iddw9van5i,ITALY,25-44,With Spouse,1.0,1.0,Widlife Tourism,Leisure and Holidays,Yes,7,...,Yes,Yes,Yes,Yes,Yes,61-90,"sunny,",7-14,NaN,0.0


In [87]:
train.isna().sum()

trip_id                           0
country                         227
age_group                         8
travel_companions               733
num_females                       2
num_males                         4
main_activity                   128
visit_purpose                     0
is_first_visit                   98
mainland_stay_nights              0
island_stay_nights                0
tour_type                         0
intl_transport_included         146
info_source                       0
accomodation_included           141
food_included                   171
domestic_transport_included       0
sightseeing_included              0
guide_included                    0
insurance_included              235
days_booked_before_trip        1547
arrival_weather                3395
total_trip_days                 402
has_special_requirements       8086
spend_category                    0
dtype: int64

In [88]:
train.isna().sum()
nullValueColumns=train.isna().sum()>0
nullValueColumns=nullValueColumns[nullValueColumns]
nullValueColumns

country                     True
age_group                   True
travel_companions           True
num_females                 True
num_males                   True
main_activity               True
is_first_visit              True
intl_transport_included     True
accomodation_included       True
food_included               True
insurance_included          True
days_booked_before_trip     True
arrival_weather             True
total_trip_days             True
has_special_requirements    True
dtype: bool

In [89]:
# i see that the null values are
train

,trip_id,country,age_group,travel_companions,num_females,num_males,main_activity,visit_purpose,is_first_visit,mainland_stay_nights,...,food_included,domestic_transport_included,sightseeing_included,guide_included,insurance_included,days_booked_before_trip,arrival_weather,total_trip_days,has_special_requirements,spend_category
0,tour_idftaa27vp,FRANCE,45-64,With Spouse and Children,1.0,2.0,Beach Tourism,Leisure and Holidays,Yes,0,...,No,No,No,No,No,NaN,"cloudy,",30+,NaN,1.0
1,tour_iduck75m57,KENYA,45-64,Alone,1.0,0.0,Conference Tourism,Meetings and Conference,Yes,6,...,No,No,No,No,No,15-30,"sunny,",30+,NaN,2.0
2,tour_id8y3w40h8,SOUTH AFRICA,25-44,With Other Friends/Relatives,2.0,0.0,Cultural Tourism,Meetings and Conference,No,4,...,No,No,No,No,No,90+,"sunny,",30+,none,2.0
3,tour_idkoh8mkgr,ITALY,25-44,With Spouse,1.0,1.0,Widlife Tourism,Leisure and Holidays,Yes,0,...,Yes,Yes,Yes,Yes,No,8-14,NaN,NaN,none,0.0
4,tour_idkmsfa00a,ITALY,25-44,With Spouse,1.0,1.0,Beach Tourism,Leisure and Holidays,Yes,0,...,Yes,No,No,No,No,90+,"sunny,",7-14,NaN,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12649,tour_idlln8s2r3,NETHERLANDS,25-44,With Other Friends/Relatives,0.0,2.0,Wildlife Tourism,Business,No,7,...,Yes,Yes,Yes,No,No,NaN,Stormy,30+,NaN,0.0
12650,tour_id3ta5n81h,KENYA,25-44,Alone,1.0,0.0,Hunting Tourism,Meetings and Conference,No,10,...,No,No,No,No,No,61-90,humid,7-14,NaN,1.0
12651,tour_id88nvd60z,UNITED STATES OF AMERICA,45-64,Alone,0.0,1.0,Widlife Tourism,Scientific and Academic,No,9,...,No,No,No,No,No,90+,"sunny,",1-6,NaN,1.0
12652,tour_iddw9van5i,ITALY,25-44,With Spouse,1.0,1.0,Widlife Tourism,Leisure and Holidays,Yes,7,...,Yes,Yes,Yes,Yes,Yes,61-90,"sunny,",7-14,NaN,0.0


In [90]:
train.drop(columns=nullValueColumns.index, inplace=True)

In [91]:
train

,trip_id,visit_purpose,mainland_stay_nights,island_stay_nights,tour_type,info_source,domestic_transport_included,sightseeing_included,guide_included,spend_category
0,tour_idftaa27vp,Leisure and Holidays,0,7,Independent,"Newspaper, magazines, brochures",No,No,No,1.0
1,tour_iduck75m57,Meetings and Conference,6,0,Independent,Others,No,No,No,2.0
2,tour_id8y3w40h8,Meetings and Conference,4,2,Independent,"Friends, relatives",No,No,No,2.0
3,tour_idkoh8mkgr,Leisure and Holidays,0,7,Package Tour,"Travel agent, tour operator",Yes,Yes,Yes,0.0
4,tour_idkmsfa00a,Leisure and Holidays,0,7,Package Tour,"Travel agent, tour operator",No,No,No,0.0
...,...,...,...,...,...,...,...,...,...,...
12649,tour_idlln8s2r3,Business,7,0,Package Tour,"Radio, TV, Web",Yes,Yes,No,0.0
12650,tour_id3ta5n81h,Meetings and Conference,10,0,Independent,"Newspaper, magazines, brochures",No,No,No,1.0
12651,tour_id88nvd60z,Scientific and Academic,9,0,Independent,"Friends, relatives",No,No,No,1.0
12652,tour_iddw9van5i,Leisure and Holidays,7,7,Package Tour,"Travel agent, tour operator",Yes,Yes,Yes,0.0


In [92]:
train["spend_category"].value_counts()

spend_category
0.0    6245
1.0    4911
2.0    1464
Name: count, dtype: int64

preprocess!!

In [93]:

X = train.drop(columns=["spend_category","trip_id"])
y = train["spend_category"]

X.head()

,visit_purpose,mainland_stay_nights,island_stay_nights,tour_type,info_source,domestic_transport_included,sightseeing_included,guide_included
0,Leisure and Holidays,0,7,Independent,"Newspaper, magazines, brochures",No,No,No
1,Meetings and Conference,6,0,Independent,Others,No,No,No
2,Meetings and Conference,4,2,Independent,"Friends, relatives",No,No,No
3,Leisure and Holidays,0,7,Package Tour,"Travel agent, tour operator",Yes,Yes,Yes
4,Leisure and Holidays,0,7,Package Tour,"Travel agent, tour operator",No,No,No


In [94]:
# ===============================================================
# Identify feature types
# ===============================================================

categorical_cols = [col for col in X.columns if X[col].dtype == "object"]

num_cols = X.select_dtypes(include=["int64","float64"]).columns.tolist()

print("Categorical:", categorical_cols)
print("Numerical:", num_cols)


Categorical: ['visit_purpose', 'tour_type', 'info_source', 'domestic_transport_included', 'sightseeing_included', 'guide_included']
Numerical: ['mainland_stay_nights', 'island_stay_nights']


In [95]:
# ===============================================================
# Multi-column LabelEncoder for pipelines
# ===============================================================

class MultiColumnLabelEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.encoders = {}

    def fit(self, X, y=None):
        for col in X.columns:
            le = LabelEncoder()
            le.fit(X[col].astype(str))
            self.encoders[col] = le
        return self

    def transform(self, X):
        X = X.copy()
        for col in X.columns:
            X[col] = self.encoders[col].transform(X[col].astype(str))
        return X


In [96]:
# ===============================================================
# Preprocessing pipeline
# ===============================================================

# preprocess = ColumnTransformer(
#     transformers=[
#         ("cat", MultiColumnLabelEncoder(), categorical_cols),
#         ("num", StandardScaler(), num_cols)
#     ],
#     remainder="passthrough"
# )
from sklearn.preprocessing import OneHotEncoder

preprocess = ColumnTransformer(
    transformers=[
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
        ("num", StandardScaler(), num_cols)
    ]
)
preprocess


,transformers,"[('cat', ...), ('num', ...)]"
,remainder,'drop'
,sparse_threshold,0.3
,n_jobs,None
,transformer_weights,None
,verbose,False
,verbose_feature_names_out,True
,force_int_remainder_cols,'deprecated'
,categories,'auto'
,drop,None
,sparse_output,True


In [97]:
# ===============================================================
# Split train/val
# ===============================================================

X_train, X_val, y_train, y_val = train_test_split(
    X, y,
    test_size=0.2,
    stratify=y,
    random_state=42
)

X_train.shape, X_val.shape


((10096, 8), (2524, 8))

In [98]:
# ===============================================================
# SVM Pipeline
# ===============================================================

svm_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("svm", SVC(kernel="rbf", C=3, gamma="scale"))
])


In [99]:
# ===============================================================
# Train SVM
# ===============================================================

svm_pipeline.fit(X_train, y_train)

svm_pred_val = svm_pipeline.predict(X_val)
svm_acc = accuracy_score(y_val, svm_pred_val)

print("SVM Validation Accuracy:", svm_acc)


SVM Validation Accuracy: 0.7321711568938193


In [100]:
train.isna().sum()

trip_id                        0
visit_purpose                  0
mainland_stay_nights           0
island_stay_nights             0
tour_type                      0
info_source                    0
domestic_transport_included    0
sightseeing_included           0
guide_included                 0
spend_category                 0
dtype: int64

In [101]:
train.columns
test[train.drop(columns=["spend_category"]).columns].isna().sum() # 

trip_id                        0
visit_purpose                  0
mainland_stay_nights           0
island_stay_nights             0
tour_type                      0
info_source                    0
domestic_transport_included    0
sightseeing_included           0
guide_included                 0
dtype: int64

In [102]:
X

,visit_purpose,mainland_stay_nights,island_stay_nights,tour_type,info_source,domestic_transport_included,sightseeing_included,guide_included
0,Leisure and Holidays,0,7,Independent,"Newspaper, magazines, brochures",No,No,No
1,Meetings and Conference,6,0,Independent,Others,No,No,No
2,Meetings and Conference,4,2,Independent,"Friends, relatives",No,No,No
3,Leisure and Holidays,0,7,Package Tour,"Travel agent, tour operator",Yes,Yes,Yes
4,Leisure and Holidays,0,7,Package Tour,"Travel agent, tour operator",No,No,No
...,...,...,...,...,...,...,...,...
12649,Business,7,0,Package Tour,"Radio, TV, Web",Yes,Yes,No
12650,Meetings and Conference,10,0,Independent,"Newspaper, magazines, brochures",No,No,No
12651,Scientific and Academic,9,0,Independent,"Friends, relatives",No,No,No
12652,Leisure and Holidays,7,7,Package Tour,"Travel agent, tour operator",Yes,Yes,Yes


In [103]:
test

,trip_id,country,age_group,travel_companions,num_females,num_males,main_activity,visit_purpose,is_first_visit,mainland_stay_nights,...,accomodation_included,food_included,domestic_transport_included,sightseeing_included,guide_included,insurance_included,days_booked_before_trip,arrival_weather,total_trip_days,has_special_requirements
0,tour_id8gzpck76,CONGO,25-44,Alone,0.0,1.0,Widlife Tourism,Business,No,14,...,No,No,No,No,No,No,15-30,"sunny,",15-30,"wheelchair,"
1,tour_idow1zxkou,SWIZERLAND,45-64,With Spouse,1.0,1.0,Widlife Tourism,Leisure and Holidays,Yes,8,...,Yes,Yes,No,No,Yes,No,61-90,Stormy,1-6,NaN
2,tour_idue7esfqz,MEXICO,45-64,With Other Friends/Relatives,2.0,0.0,Cultural Tourism,Leisure and Holidays,Yes,3,...,Yes,Yes,Yes,Yes,Yes,No,1-7,Stormy,30+,none
3,tour_idnj3mjzpb,JAPAN,25-44,With Other Friends/Relatives,1.0,1.0,Widlife Tourism,Leisure and Holidays,Yes,5,...,Yes,Yes,Yes,Yes,Yes,No,90+,"sunny,",7-14,NaN
4,tour_ida3us5yk2,SPAIN,25-44,With Other Friends/Relatives,2.0,0.0,Wildlife Tourism,Leisure and Holidays,Yes,0,...,Yes,Yes,No,Yes,No,Yes,61-90,Stormy,30+,"dietary needs,"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5847,tour_id0bnf9xcp,ZIMBABWE,25-44,Alone,0.0,1.0,Beach Tourism,Business,Yes,3,...,No,No,No,No,No,No,NaN,other,1-6,NaN
5848,tour_idqfcw3991,UNITED STATES OF AMERICA,25-44,With Spouse,1.0,1.0,Widlife Tourism,Leisure and Holidays,Yes,9,...,Yes,Yes,Yes,Yes,Yes,No,31-60,"sunny,",30+,NaN
5849,tour_idrarc7v2s,SWEDEN,25-44,With Spouse and Children,2.0,1.0,Beach Tourism,Leisure and Holidays,Yes,0,...,No,No,No,No,No,No,61-90,NaN,30+,NaN
5850,tour_idvuu6dqrf,PAKISTAN,25-44,Alone,0.0,1.0,Hunting Tourism,Visiting Friends and Relatives,Yes,2,...,No,No,No,No,No,No,61-90,"cloudy,",7-14,NaN


In [104]:
# ===============================================================
# SVM Submission
# ===============================================================

svm_test_pred = svm_pipeline.predict(test)

sub_svm = pd.DataFrame({
    "trip_id": test["trip_id"],
    "spend_category": svm_test_pred
})

sub_svm.to_csv("svm_submission.csv", index=False)

sub_svm.head()


,trip_id,spend_category
0,tour_id8gzpck76,1.0
1,tour_idow1zxkou,0.0
2,tour_idue7esfqz,0.0
3,tour_idnj3mjzpb,0.0
4,tour_ida3us5yk2,0.0


In [46]:
# ===============================================================
# Build Keras model for pipeline
# ===============================================================

def create_model():
    model = Sequential([
        Dense(256, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(128, activation='relu'),
        BatchNormalization(),
        Dropout(0.3),

        Dense(64, activation='relu'),
        BatchNormalization(),

        Dense(3, activation='softmax')
    ])

    model.compile(
        optimizer="adam",
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model


In [47]:
# ===============================================================
# Neural Network Pipeline
# ===============================================================

nn_pipeline = Pipeline(steps=[
    ("preprocess", preprocess),
    ("nn", KerasClassifier(
        build_fn=create_model,
        epochs=25,
        batch_size=256,
        verbose=1
    ))
])


NameError: name 'KerasClassifier' is not defined

In [ ]:
# ===============================================================
# Train NN
# ===============================================================

nn_pipeline.fit(X_train, y_train)

nn_pred_val = nn_pipeline.predict(X_val)
nn_acc = accuracy_score(y_val, nn_pred_val)

print("Neural Network Validation Accuracy:", nn_acc)


In [ ]:
# ===============================================================
# NN Submission
# ===============================================================

nn_test_pred = nn_pipeline.predict(test)

sub_nn = pd.DataFrame({
    "trip_id": test["trip_id"],
    "spend_category": nn_test_pred
})

sub_nn.to_csv("nn_submission.csv", index=False)

sub_nn.head()
